In [2]:
#import libraries and set client_id
import requests
import json
import urllib
import MAL_Cookbook

client_id = "0bd76a18d2f4d2bb27ca27105be29188"

base_url = "https://api.myanimelist.net/v2/"
auth_header = {"X-MAL-CLIENT-ID" : client_id}



In [7]:
#Function where you give a MAL username and it returns the users top 5 most watched genres

def topGenres(username):
    genres = MAL_Cookbook.get_genre_preferences(client_id, username)
    
#add each genre to the genreList as a tuple (genre, numer of shows in that genre)
    genreList = []
    for index in genres:
       # print(genres[index]['genre']['name'], genres[index]['count'])
        genreList.append(tuple([genres[index]['genre']['name'], genres[index]['count']]))

    gl = genreList.sort(key = lambda x: -x[1]) 

    #sort the genre and return the top 5
    
    TopG = []
    for gen in genreList:
        TopG.append(gen[0])                  
    return(TopG[:5])

#Function that you input a MAL username and it will return the median year of the shows they watched

def medianYear(username):
    yl = MAL_Cookbook.get_vintage_preferences(client_id, username)

    yearList = []
    medianlist = []
    for index in yl:
      #  print(yl[index]['count'])
       # print(yl[index]['season']['year'])
        times = yl[index]['count']
        #Add each show year to a list
        while times > 0:
            medianlist.append(yl[index]['season']['year'])
            times = times - 1
       # print(genres[index]['genre']['name'], genres[index]['count'])
       
    #sort the list 
    medianlist.sort()
   # return the median
    return(medianlist[int(len(medianlist)/2)])

#function that you input a MAL username and a desired score
#The function will return all show id's on a MAL user list above the entered score  

def topShowsOnList(username, rat):
    rating = []
    animelist = MAL_Cookbook.get_user_list_ids(client_id, username)
    for a in animelist:
        for al in animelist[a]:
            score = al['list_status']['score']
            sid = al['node']['id']
            if score >= rat:
                rating.append(sid)
    return(rating)

#Returns the median of the length of each show on a users list
def medianNumEpisodes(username):
    listOfShows = topShowsOnList(username, 0)
    listOfNums = []
    #def get_show_stats(client_id, show_id=-1, show_title=-1, content_type='anime', stat='all'):
    for i in listOfShows:
        num = int(MAL_Cookbook.get_show_stats(client_id, show_id=i, stat='num_episodes'))
        if num > 0:
            listOfNums.append(num)

    listOfNums.sort()
    return(listOfNums[int(len(listOfNums)/2)])

In [8]:

#This block of code import the top shows from MAL and puts them in storedTopShows as a tuple with (showId,showyear,genres) 
import time
topshows = MAL_Cookbook.get_top_shows(client_id, 100)
storedTopShows = []
h = 0
for showId in topshows:
    h = h + 1
    showyear = MAL_Cookbook.get_show_stats(client_id, show_id=showId, content_type='anime', stat='start_season')
    genres = MAL_Cookbook.get_genre(client_id, show_ids=showId, content_type='anime')
    print(h)    
    storedTopShows.append(tuple([showId,showyear,genres]))
    if h % 50 == 0:
        time.sleep(45)

NameError: name 'get_top_shows' is not defined

In [9]:
#This block of code will get information on a user that you select

username = "happydr"

my = medianYear(username)
print(my)
tg = topGenres(username)
print(tg)
me = medianNumEpisodes(username)
print(me)
userratedshows = topShowsOnList(username, 8)
print(len(userratedshows))
userIdList = topShowsOnList(username, 0)
print(len(userIdList))

2019
['Fantasy', 'Action', 'Isekai', 'Adventure', 'Comedy']
12


In [29]:


#This section of code will create the recommendations
cc = 0
rScores = []
#go through ever show in the top
for showId in storedTopShows:
    currentid = showId[0]
    showyear = showId[1]
    genres = showId[2]
    #showyear = get_show_stats(client_id, show_id=showId, content_type='anime', stat='start_season')
    #genres = get_genre(client_id, show_ids=showId, content_type='anime')

    #create a year score by taking away 10% for each year away from median
    yearScore = 10 - abs(my - showyear['year'])
    if yearScore < 0:
        yearScore = 0
   # print("Year Score", yearScore)

#create genrescore by adding points for each genre a show has in a users top genres
    genreScore = 0
    y = 0
    for i in genres:
        while y < len(genres[i]):
            g = genres[i][y]['name']
            if g in tg:
                ind = tg.index(g)
                genreScore = genreScore + 5 - ind
                #print(5-ind)
            y = y + 1
    #print("OGgenreScore", genreScore)

    genreScore = genreScore / 15
    genreScore = genreScore * 10

    #print("genreScore", genreScore)
    
    #gives a point if a topshow is a user recommended show on mal
    userRecScore = 0
    if currentid in userratedshows:
        userRecScore = 10
        #print(userRecScore)
    #print(currentid)
    
    #Calculates the shows rating with different weights
    rPoint = (yearScore * 0.15) + (genreScore * .65) + (userRecScore *  .20)
    
   # print(showId, rPoint)
    rScores.append(tuple([showId[0], rPoint]))
    cc = cc + 1
   # print(cc)

#Removes shows you have watched
sanScores = []
f = 0
for s in rScores:
    if s[0] in userIdList:
        f = f + 1
    else:
        sanScores.append(s)
#print(f)
rwww = sanScores.sort(key = lambda x: -x[1]) 

#Prints your top 10 shows
c = 0
while c < 10:
    show = MAL_Cookbook.get_show_stats(client_id, show_id=sanScores[c][0], content_type='anime')
    print(c + 1, show['title'], sanScores[c])
    c = c + 1

1 Fate/stay night Movie: Heaven's Feel - II. Lost Butterfly (33049, 5.4)
2 Fate/stay night Movie: Heaven's Feel - III. Spring Song (33050, 5.25)
3 Mononoke Hime (164, 4.766666666666667)
4 Made in Abyss Movie 3: Fukaki Tamashii no Reimei (36862, 4.383333333333334)
5 Fate/Zero 2nd Season (11741, 4.3500000000000005)
6 Ousama Ranking (40834, 4.233333333333333)
7 Made in Abyss (34599, 4.233333333333333)
8 JoJo no Kimyou na Bouken Part 5: Ougon no Kaze (37991, 3.95)
9 Mushishi Zoku Shou: Suzu no Shizuku (28957, 3.9333333333333336)
10 Mushishi Zoku Shou 2nd Season (24701, 3.7833333333333337)
